# Collect RAWS Data with SynopticPy

## Setup

In [1]:
import matplotlib.pyplot as plt
from datetime import datetime

from synoptic.services import stations_timeseries

C:\Users\jhirs\anaconda3\lib\site-packages\synoptic\accessors.py:28: UserWarning: map making not available without Brian's cartopy_tools
  warnings.warn("map making not available without Brian's cartopy_tools")
C:\Users\jhirs\anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


## Station Time Series

In [2]:
params = dict(
    stid=["NWRU1"],
    vars=["air_temp", "relative_humidity", "precip_accum", "fuel_moisture", "wind_speed", "solar_radiation"],
    start=datetime(2023, 8, 15),
    end=datetime(2023, 8, 18),
)

a = stations_timeseries(verbose="HIDE", **params)

a


 🚚💨 Speedy Delivery from Synoptic API [timeseries]: https://api.synopticdata.com/v2/stations/timeseries?stid=NWRU1&vars=air_temp,relative_humidity,precip_accum,fuel_moisture,wind_speed,solar_radiation&start=202308150000&end=202308180000&token=🙈HIDDEN



,air_temp,fuel_moisture,precip_accum,relative_humidity,solar_radiation,wind_speed
date_time,,,,,,
2023-08-15 00:04:00+00:00,26.111,8.3,374.142,26.0,336.0,1.343
2023-08-15 01:04:00+00:00,25.556,8.0,374.142,27.0,314.0,1.343
2023-08-15 02:04:00+00:00,22.222,7.9,374.142,38.0,109.0,0.895
2023-08-15 03:04:00+00:00,16.111,8.4,374.142,60.0,3.0,0.000
2023-08-15 04:04:00+00:00,14.444,9.1,374.142,66.0,0.0,0.000
...,...,...,...,...,...,...
2023-08-17 19:04:00+00:00,24.444,10.2,394.970,46.0,398.0,0.000
2023-08-17 20:04:00+00:00,16.111,11.3,395.732,83.0,302.0,0.895
2023-08-17 21:04:00+00:00,17.222,17.7,395.732,83.0,83.0,0.895


In [3]:
type(a)

pandas.core.frame.DataFrame

In [4]:
print(f'Station Elevation: {a.attrs["ELEVATION"]}')
print(f'Lat: {a.attrs["latitude"]}')
print(f'Lon: {a.attrs["longitude"]}')
print(f'Temp Units: {a.attrs["UNITS"]["air_temp"]}')

Station Elevation: 8280
Lat: 40.6125
Lon: -111.09833
Temp Units: Celsius


In [5]:
a.attrs

{'ID': 1496,
 'STID': 'NWRU1',
 'NAME': 'NORWAY',
 'ELEVATION': 8280,
 'STATUS': 'ACTIVE',
 'MNET_ID': 2,
 'STATE': 'UT',
 'TIMEZONE': 'America/Denver',
 'ELEV_DEM': 8251.3,
 'PERIOD_OF_RECORD': {'start': '1998-07-28T00:00:00Z',
  'end': '2024-02-12T20:04:00Z'},
 'UNITS': {'position': 'm',
  'elevation': 'ft',
  'precip_accum': 'Millimeters',
  'solar_radiation': 'W/m**2',
  'air_temp': 'Celsius',
  'fuel_moisture': 'gm',
  'relative_humidity': '%',
  'wind_speed': 'm/s'},
 'QC_FLAGGED': False,
 'RESTRICTED': False,
 'RENAMED': {'air_temp_set_1': 'air_temp',
  'fuel_moisture_set_1': 'fuel_moisture',
  'precip_accum_set_1': 'precip_accum',
  'relative_humidity_set_1': 'relative_humidity',
  'solar_radiation_set_1': 'solar_radiation',
  'wind_speed_set_1': 'wind_speed'},
 'latitude': 40.6125,
 'longitude': -111.09833,
 'SUMMARY': {'NUMBER_OF_OBJECTS': 1,
  'RESPONSE_CODE': 1,
  'RESPONSE_MESSAGE': 'OK',
  'METADATA_RESPONSE_TIME': '415.1 ms',
  'DATA_QUERY_TIME': '13.7 ms',
  'QC_QUERY_T